In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({'Size': ['small', 'medium', 'large']})
df.Size = df.Size.astype('category')

In [10]:
df.Size.cat.categories = df.Size.cat.reorder_categories(new_categories=['small', 'medium', 'large'])

In [11]:
df.Size.cat.as_ordered()

0     large
1    medium
2     small
Name: Size, dtype: category
Categories (3, object): ['small' < 'medium' < 'large']

In [14]:
print(pd.get_dummies(df.Size))

   small  medium  large
0      0       0      1
1      0       1      0
2      1       0      0


In [18]:
df.Size = df.Size.cat.codes

AttributeError: Can only use .cat accessor with a 'category' dtype

In [19]:
print(df)

   Size
0     2
1     1
2     0
